## CRAWL dari KOMPAS

### Def untuk crawling dari Portal KOMPAS
Note : 
- ganti User-Agent pada headers -> ke user agent browser sendiri

In [1]:
import os
import time
import csv
import requests
from bs4 import BeautifulSoup

def kompas_batch(keywords, n_berita=200):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
    }

    save_dir = r"/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/DATA/corpus/kompas"
    os.makedirs(save_dir, exist_ok=True)

    for keyword in keywords:
        print(f"/n🚀 Mulai scraping: {keyword}")
        data_kompas = []
        page = 1

        while len(data_kompas) < n_berita:
            alamatURL = f'https://search.kompas.com/search?q={keyword}&sortby=time&page={page}'
            req = requests.get(alamatURL, headers=headers)
            print(f"🔄 Halaman {page} untuk {keyword}")

            if req.status_code != 200:
                print("❌ Gagal akses halaman.")
                break

            # pakai parser fallback kalau lxml error
            try:
                soup = BeautifulSoup(req.content, 'lxml')
            except Exception:
                soup = BeautifulSoup(req.content, 'html.parser')

            articles = soup.find_all('div', class_='articleItem')
            if not articles:
                print("❌ Tidak ada artikel lagi.")
                break

            for article in articles:
                try:
                    a_tag = article.find('a', href=True)
                    if not a_tag:
                        continue

                    url = a_tag['href']
                    title_tag = article.find('h2', class_='articleTitle')
                    judul = title_tag.get_text(strip=True) if title_tag else a_tag.get_text(strip=True)

                    # buka halaman detail
                    detail_req = requests.get(url, headers=headers)
                    try:
                        detail_soup = BeautifulSoup(detail_req.content, 'lxml')
                    except Exception:
                        detail_soup = BeautifulSoup(detail_req.content, 'html.parser')

                    body = detail_soup.find('div', class_='read__content')
                    berita = body.get_text(separator=' ', strip=True) if body else "Tidak ditemukan"

                    if berita and len(berita) > 30:
                        data_kompas.append({
                            'judul': judul,
                            'berita': berita,
                            'url': url,
                        })
                        print(f"[{len(data_kompas)}] {judul}")

                        if len(data_kompas) >= n_berita:
                            break

                    time.sleep(1)  # delay biar ga ke-block

                except Exception as e:
                    print("⚠ Error ambil artikel:", e)
                    continue

            page += 1

        # Simpan hasil per keyword
        if data_kompas:
            filename = os.path.join(save_dir, f"kompas_news_{keyword.lower().replace(' ','')}_{time.strftime('%Y%m%d_%H%M%S')}.csv")
            with open(filename, "w", newline="", encoding="utf-8-sig") as f:
                writer = csv.DictWriter(f, fieldnames=["judul", "berita", "url"])
                writer.writeheader()
                writer.writerows(data_kompas)

            print(f"/n✅ {len(data_kompas)} artikel '{keyword}' disimpan ke {filename}")
        else:
            print(f"❌ Tidak ada data valid untuk '{keyword}'")



In [ ]:
keywords = [
    "BI rate",
    "Pertumbuhan ekonomi",
    "Inflasi",
    "Bank Indonesia",
    "Perbankan digital",
    "IHSG",
    "Pelemahan Rupiah",
    "Investasi",
    "Ekonomi Indonesia",
    "Danantara",
    "OJK",
    "Reksadana",
    "Bursa Efek Indonesia",
    "BEI",
    "Ekonomi digital"
]

kompas_batch(keywords, n_berita=1000)


/n🚀 Mulai scraping: BI rate
🔄 Halaman 1 untuk BI rate
[1] BI Rate Turun, OJK Imbau Perbankan Sesuaikan Tingkat Suku Bunga
[2] BI Rate Turun, Perbankan Ikut Pangkas Bunga Simpanan
[3] Prospek Saham BBCA di Tengah Penurunan BI Rate
[4] Penempatan Dana Pemerintah Rp 200 T dan Pemangkasan BI Rate Jadi Bahan Bakar Baru Pertumbuhan Ekonomi
[5] Ekonom Prediksi BI Rate Bertahan di 5 Persen Pada September 2025, Ini Alasannya
[6] Rp 200 T Mengalir, BI Rate 4,75 Persen: Di Persimpangan Rupiah dan Kredit Murah
[7] Bank Indonesia Turunkan Suku Bunga Acuan untuk Kelima Kalinya, BI Rate Kini 4,75 Persen
[8] Rate Halte TJ yang Paling Bikin Ngos-Ngosan
[9] Kendala Pemasangan Lampu Bi-LED dan Solusinya
[10] BI: Pertumbuhan Ekonomi Indonesia Perlu Semakin Ditingkatkan
[11] Kenapa Rupiah Melemah? Ini Penjelasan Bos BI
[12] Bos BI Jelaskan Penyebab Rupiah Melemah
[13] Cara Mudah Memperbaiki Skor BI Checking, Resmi OJK
[14] Candaan Purbaya Saat Jumpa Bos BI, Diplomasi Bebek Goreng
[15] Tips Memilih Lampu Bi

### REPUBLIKA (sementara)

In [8]:
import os
import time
import csv
import json
from urllib.parse import quote
from bs4 import BeautifulSoup

# Import library Selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

def republika(keywords, n_berita):
    # --- Konfigurasi Selenium ---
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Menjalankan browser di background (tanpa window)
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36")

    # Secara otomatis menginstal dan menyiapkan driver untuk Chrome
    service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    print("✅ Driver Chrome headless berhasil diinisialisasi.")

    save_dir = r"/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/DATA/corpus/republika"
    os.makedirs(save_dir, exist_ok=True)

    for keyword in keywords:
        print(f"\n🚀 Mulai scraping untuk kata kunci: '{keyword}'")
        data_republika = []
        page = 1
        
        while len(data_republika) < n_berita:
            api_url = f'https://republika.co.id/search/v3/q?q={quote(keyword)}&sortby=time&page={page}'
            
            try:
                print(f"🔄 Browser membuka API halaman {page} untuk '{keyword}'...")
                
                # === PERUBAHAN UTAMA: Menggunakan Selenium untuk membuka URL API ===
                driver.get(api_url)
                time.sleep(2) # Beri waktu browser untuk memproses

                # Browser akan menampilkan JSON mentah di dalam tag <pre>
                # Kita ambil teks dari sana
                pre_tag = driver.find_element(By.TAG_NAME, 'pre')
                json_text = pre_tag.text
                
                # Parsing teks JSON secara manual
                search_results = json.loads(json_text)
                articles = search_results.get('articles', [])

            except Exception as e:
                print(f"❌ Gagal mengambil atau parsing data dari browser: {e}")
                break

            if not articles:
                print("✅ Tidak ada artikel lagi dari API.")
                break

            for article_data in articles:
                try:
                    url = article_data.get('url')
                    judul = article_data.get('title', 'Tanpa Judul')
                    if not url:
                        continue

                    # Mengambil halaman detail menggunakan Selenium juga
                    driver.get(url)
                    time.sleep(1) # Beri waktu halaman untuk render
                    
                    # Gunakan BeautifulSoup untuk mem-parsing source code yang sudah dirender oleh browser
                    detail_soup = BeautifulSoup(driver.page_source, 'html.parser')
                    konten = detail_soup.find('div', class_='post-content')
                    
                    if konten:
                        for tag in konten.find_all(['strong', 'em'], text=lambda t: t and 'baca juga' in t.lower()):
                            tag.find_parent('p').decompose()
                        isi = konten.get_text(separator=' ', strip=True)
                    else:
                        isi = "Tidak ditemukan isi berita"

                    if len(isi) > 50:
                        data_republika.append({
                            'judul': judul,
                            'berita': isi,
                            'url': url,
                        })
                        print(f"    [{len(data_republika)}] Berhasil mengambil: {judul}")

                    if len(data_republika) >= n_berita:
                        break
                    
                except Exception as e:
                    print(f"    ⚠️ Error saat memproses artikel {url}: {e}")
                    continue
            
            page += 1
            if len(data_republika) >= n_berita:
                break

        if data_republika:
            filename = os.path.join(save_dir, f"republika_news_{keyword.lower().replace(' ','_')}_{time.strftime('%Y%m%d')}.csv")
            with open(filename, "w", newline="", encoding="utf-8-sig") as f:
                writer = csv.DictWriter(f, fieldnames=["judul", "berita", "url"])
                writer.writeheader()
                writer.writerows(data_republika)
            print(f"\n✅ Selesai! {len(data_republika)} artikel '{keyword}' disimpan ke {filename}")
        else:
            print(f"❌ Tidak ada data yang ditemukan untuk '{keyword}'")

    # Tutup browser setelah semua pekerjaan selesai
    driver.quit()
    print("\n✅ Browser ditutup. Semua pekerjaan selesai.")

In [9]:
keywords = [
    "BI rate",
    "Pertumbuhan ekonomi",
    "Inflasi",
    "Bank Indonesia",
    "Perbankan digital",
    "IHSG",
    "Pelemahan Rupiah",
    "Investasi",
    "Ekonomi Indonesia",
    "Danantara",
    "OJK",
    "Reksadana",
    "Bursa Efek Indonesia",
    "BEI",
    "Ekonomi digital"
]

republika(keywords, n_berita=10)


✅ Driver Chrome headless berhasil diinisialisasi.

🚀 Mulai scraping untuk kata kunci: 'BI rate'
🔄 Browser membuka API halaman 1 untuk 'BI rate'...


KeyboardInterrupt: 